In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from typing import Tuple, Optional

In [ ]:
self.df_predictions = pd.read_csv(f"{self.path}/data/cleaned/predictions.csv")
self.df_inventory = pd.read_excel(f"{self.path}/data/raw/catusita/inventory.xlsx")
self.df_rfm = pd.read_csv(f"{self.path}/data/process/df_rfm.csv")
self.df_tc = pd.read_excel(f"{self.path}/data/raw/catusita/saldo de todo 04.11.2024.2.xls", skiprows=2)
self.df_products = pd.read_csv(f"{self.path}/data/process/catusita_consolidated.csv")
try:
    self.df_backorder = pd.read_excel(f"{self.path}/data/raw/catusita/backorder12_12.xlsx")
except FileNotFoundError:
    self.df_backorder = pd.DataFrame()

### to_datime
self.df_tc['Ult. Fecha'] = pd.to_datetime(self.df_tc['Ult. Fecha'], errors='coerce')
self.df_products['fecha'] = pd.to_datetime(self.df_products['fecha'])
self.df_predictions['date'] = pd.to_datetime(self.df_predictions['date'])
self.df_inventory['FECHA AL'] = pd.to_datetime(self.df_inventory['FECHA AL'], format='%Y%m%d')

### processing data for raw tables
## df_tc
self.df_tc = self.df_tc[['Código','Mnd','Fob','Ult. Fecha','Ult. Compra']]
self.df_tc.columns = ['codigo', 'moneda', 'monto', 'ultima_fecha', 'ultima_compra']
self.df_tc['codigo'] = self.df_tc['codigo'].astype(str)
self.df_tc = self.df_tc.dropna(subset=['ultima_fecha'])
self.df_tc['codigo'] = self.df_tc['codigo'].str.lower()
self.df_tc = self.df_tc[self.df_tc['ultima_fecha'].notna()]

## df_product
self.df_products['fecha_mensual'] = self.df_products['fecha'].dt.to_period('M').dt.to_timestamp()       
# crear variable precio
self.df_products['precio'] = self.df_products['venta_pen'] / self.df_products['cantidad']
# crear variable margen
self.df_products['margen'] = self.df_products['venta_pen'] / self.df_products['costo'] - 1
self.df_margin_result = self.df_products.groupby('articulo').agg(
    total_venta_pen=('venta_pen', 'sum'),
    mean_margen=('margen', 'mean')
).reset_index().sort_values(by='total_venta_pen', ascending=False)
# agregar por fecha_mensual, articulo, fuente_suministro 
self.df_products = self.df_products.groupby(['fecha_mensual', 'articulo', 'fuente_suministro']).agg({
    'codigo': 'first', 
    'cantidad': 'sum',
    'transacciones': 'sum',
    'venta_pen': 'sum', 
    'costo': 'mean',
    'precio': 'mean',
    'lt': 'first'
}).reset_index().rename(columns={'venta_pen': 'total_venta_pen','margen': 'mean_margen'})

## df_predictions
self.df_predictions = self.df_predictions.rename(columns={'sku': 'articulo'})

## df_inventory
self.df_inventory.columns = ['cia', 'date', 'codigo', 'descripcion', 'um', 'stock']
self.df_inventory.loc[:, 'codigo'] = self.df_inventory['codigo'].str.lower()
self.df_inventory = self.df_inventory.groupby(['date','codigo','descripcion','um']).agg(
    {
        'stock':'sum'
    }
).reset_index()

In [3]:
tipo_de_cambio_df = tipo_de_cambio_df[['Código','Mnd','Fob','Ult. Fecha','Ult. Compra']]
tipo_de_cambio_df.columns = ['codigo', 'moneda', 'monto', 'ultima_fecha', 'ultima_compra']
tipo_de_cambio_df = tipo_de_cambio_df.copy()
tipo_de_cambio_df['codigo'] = tipo_de_cambio_df['codigo'].astype(str)
tipo_de_cambio_df = tipo_de_cambio_df.dropna(subset=['ultima_fecha'])
tipo_de_cambio_df['codigo'] = tipo_de_cambio_df['codigo'].str.lower()
tipo_de_cambio_df = tipo_de_cambio_df[tipo_de_cambio_df['ultima_fecha'].notna()]
tipo_de_cambio_df['ultima_fecha'] = pd.to_datetime(tipo_de_cambio_df['ultima_fecha'], errors='coerce')

In [4]:
start = tipo_de_cambio_df['ultima_fecha'].min().date()
end = tipo_de_cambio_df['ultima_fecha'].max().date()
currency_pairs = ['PENUSD=X', 'EURUSD=X', 'JPYUSD=X', 'GBPUSD=X']
data = yf.download(currency_pairs, start=start, end=end)
closing_prices = data['Close']
closing_prices.columns = [col.split('.')[0] for col in closing_prices.columns]

[*********************100%***********************]  4 of 4 completed


In [5]:
long_format = closing_prices.reset_index().melt(id_vars='Date', var_name='Currency Pair', value_name='Closing Price')
long_format['Currency Pair'] = long_format['Currency Pair'].str.replace('=X', '', regex=False)
long_format = long_format.dropna(subset=['Closing Price'])

full_date_range = pd.date_range(start=long_format['Date'].min(), end=long_format['Date'].max(), freq='D')
currency_pairs = long_format['Currency Pair'].unique()
complete_index = pd.MultiIndex.from_product([full_date_range, currency_pairs], names=['Date', 'Currency Pair'])
df_full = pd.DataFrame(index=complete_index).reset_index()

long_format = df_full.merge(long_format, on=['Date', 'Currency Pair'], how='left')
long_format['Closing Price'] = long_format.groupby('Currency Pair')['Closing Price'].fillna(method='ffill')
long_format = long_format.rename(columns={'Closing Price': 'tc'})

C:\Users\YOGA\AppData\Local\Temp\ipykernel_16572\2428493962.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  long_format['Closing Price'] = long_format.groupby('Currency Pair')['Closing Price'].fillna(method='ffill')


In [6]:
merged_df_tc = pd.merge(tipo_de_cambio_df, long_format, left_on='ultima_fecha', right_on='Date', how='left')
merged_df_tc['monto'] = pd.to_numeric(merged_df_tc['monto'], errors='coerce')
merged_df_tc['tc'] = pd.to_numeric(merged_df_tc['tc'], errors='coerce')

def convert_to_usd(row):
    if pd.isna(row['Currency Pair']) or row['moneda'] == 'USD':
        return row['monto']
    currency_pair_map = {'SOL': 'PENUSD', 'EUR': 'EURUSD', 'JPY': 'JPYUSD', 'GBP': 'GBPUSD'}
    if row['moneda'] in currency_pair_map and row['Currency Pair'] == currency_pair_map[row['moneda']]:
        return row['monto'] / row['tc'] if row['moneda'] == 'SOL' else row['monto'] * row['tc']
    return 0

merged_df_tc['monto_usd'] = merged_df_tc.apply(convert_to_usd, axis=1)
merged_df_tc = merged_df_tc[merged_df_tc['monto_usd'] != 0]
merged_df_tc_final = merged_df_tc[['codigo', 'ultima_fecha', 'monto_usd', 'ultima_compra']]
merged_df_tc_final = merged_df_tc_final[merged_df_tc_final['monto_usd'].notna()]

In [4]:
df_inventory = pd.read_excel("data/raw/catusita/inventory.xlsx")
df_inventory.columns = ['cia', 'date', 'codigo', 'descripcion', 'um', 'stock']
df_inventory.loc[:, 'codigo'] = df_inventory['codigo'].str.lower()
df_inventory['date'] = pd.to_datetime(df_inventory['date'], format='%Y%m%d')
max_date = df_inventory['date'].max()
df_inventory = df_inventory.groupby(['date','codigo','descripcion','um']).agg(
    {
        'stock':'sum'
    }
).reset_index()

df_inventory = df_inventory[
    (df_inventory['date'] != 'Periodo') & 
    (df_inventory['date'].notna())&
    (df_inventory['date']==max_date)
]
df_inventory_final=pd.concat(
    [
        pd.DataFrame(df_inventory['codigo'].unique(), columns=['codigo']),
        pd.DataFrame(df_products['articulo'].unique(), columns=['codigo'])
    ], 
    ignore_index=True
).drop_duplicates()
df_inventory_final = df_inventory_final.merge(
    df_inventory[['date','codigo','stock']].drop_duplicates(),
    how='left',
    on='codigo'
)
df_inventory_final['stock']=df_inventory_final['stock'].fillna(0)
df_inventory_final['date'] = df_inventory_final['date'].fillna(max_date)

In [ ]:
####################################################

In [22]:
articulo_mas_venta = df_products.groupby('articulo').agg({'venta_pen': 'sum'}) \
    .sort_values('venta_pen', ascending=False) \
    .reset_index() \
    .iloc[0]['articulo']



In [ ]:
###################################

In [8]:
lista_predicciones = results_models_comparison['sku'].unique()
lista_ventas = df_products['articulo'].unique()

set_predicciones = set(lista_predicciones)
set_ventas = set(lista_ventas)
set_ventas - set_predicciones

{'01277'}

In [129]:
df_merged = results_models_comparison.copy()
df_merged = df_merged.rename(columns={'sku':'articulo'})
df_merged = df_merged.merge(
    df_products[['articulo', 'fuente_suministro','lt']].drop_duplicates(), 
    how='left', 
    on='articulo'
)
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_inventory['date'] = pd.to_datetime(df_inventory['date'])
df_merged = df_merged.merge(
    df_inventory[['codigo', 'stock', 'date']].drop_duplicates(), 
    how='left', 
    left_on=['articulo', 'date'], 
    right_on=['codigo', 'date']
)
df_merged['stock'] = df_merged['stock'].fillna(0)
df_merged = df_merged.drop(columns='codigo')

In [8]:
df_precio = df_products[['articulo', 'cantidad', 'venta_pen', 'fecha']].copy()
df_precio['fecha'] = pd.to_datetime(df_precio['fecha'], errors='coerce')
# df_precio = df_precio[df_precio['fecha'].dt.year == 2024]
df_precio['precio'] = df_precio['venta_pen'] / df_precio['cantidad']
result_precio = df_precio.groupby('articulo').agg(precio=('precio', 'mean')).reset_index()

In [9]:
df_margen = df_products[['articulo', 'fuente_suministro','costo', 'venta_pen', 'fecha']].copy()
df_margen['fecha'] = pd.to_datetime(df_margen['fecha'], errors='coerce')
# df_margen = df_margen[df_margen['fecha'].dt.year == 2024]
df_margen['margen'] = df_margen['venta_pen'] / df_margen['costo'] - 1
margin_result = df_margen.groupby('articulo').agg(
    total_venta_pen=('venta_pen', 'sum'),
    mean_margen=('margen', 'mean')
).reset_index().sort_values(by='total_venta_pen', ascending=False)
margin_result_fuente = df_margen.groupby('articulo').agg(
    mean_margen=('margen', 'mean')
).reset_index()

In [11]:
df1 = df_merged[['fuente_suministro', 'date', 'articulo','real', 'catusita', 'caa','lt_x']].copy()
df1 = df1.rename(columns={'catusita': 'venta_sin_recomendacion', 'caa': 'venta_con_recomendacion'})
df1_final = df1.merge(result_precio, how='left', on='articulo')
df1_final = df1_final[['fuente_suministro', 'date', 'articulo', 'venta_sin_recomendacion', 'venta_con_recomendacion','real', 'precio','lt_x']]

df1_final['ingreso_sin_recomendacion'] = np.where(
    df1_final['venta_sin_recomendacion'] < df1_final['real'],
    df1_final['venta_sin_recomendacion'] * df1_final['precio'],
    df1_final['real'] * df1_final['precio']
)

df1_final['venta_con_recomendacion'] = np.where(
    df1_final['venta_con_recomendacion'] < df1_final['real'],
    df1_final['venta_con_recomendacion'] * df1_final['precio'],
    df1_final['real'] * df1_final['precio']
)

df1_final['ingreso_sin_recomendacion_ajustado'] = df1_final['ingreso_sin_recomendacion'] / (df1_final['lt_x'] * 0.83)
df1_final['ingreso_con_recomendación_ajustado'] = df1_final['venta_con_recomendacion'] / (df1_final['lt_x'] * 0.83)

penusd_tc = long_format[long_format['Currency Pair'] == 'PENUSD'].groupby('Date')['tc'].last().reset_index()
df1_final = df1_final.merge(penusd_tc, how='left', left_on='date', right_on='Date')
df1_final['tc'] = 1/df1_final['tc']
df1_final['ingreso_usd_sin_recomendacion'] = df1_final['ingreso_sin_recomendacion_ajustado'] / df1_final['tc']
df1_final['ingreso_usd_con_recomendacion'] = df1_final['ingreso_con_recomendación_ajustado'] / df1_final['tc']
df1_final = df1_final[['fuente_suministro', 'date', 'articulo', 'lt_x', 'ingreso_usd_sin_recomendacion', 'ingreso_usd_con_recomendacion', 'tc']]
# df1_final = df1_final.drop_duplicates()


NameError: name 'df_merged' is not defined

In [ ]:
# last_date = df_merged['date'].max()
# df_merged_last = df_merged[df_merged['date'] == last_date].copy()
df_merged_last = df_merged.copy()

df_merged_last['demanda_mensual'] = df_merged_last['caa'] / df_merged_last['lt_x']
dffinal2 = df_merged_last[['articulo', 'stock', 'caa', 'demanda_mensual', 'corr_sd', 'lt_x']]
dffinal2 = dffinal2.copy()
dffinal2['meses_proteccion'] = dffinal2['corr_sd'] / dffinal2['demanda_mensual']
dffinal2 = dffinal2[['articulo', 'stock', 'caa', 'demanda_mensual', 'meses_proteccion', 'lt_x']]
dffinal2 = dffinal2.merge(margin_result[['articulo', 'mean_margen']], how='left', on='articulo')
dffinal2 = dffinal2.merge(merged_df_tc_final, how='left', left_on='articulo', right_on='codigo')


In [ ]:
df_predicciones = pd.read_csv(f"{path}/data/cleaned/predictions.csv")
df_inventory2 = pd.read_excel(f"{path}/data/raw/catusita/inventory.xlsx")

df_predicciones = df_predicciones.rename(columns={'sku': 'articulo'})

df_inventory2 = df_inventory2[
    (df_inventory2['FECHA AL'] != 'Periodo') & 
    (df_inventory2['FECHA AL'].notna())
]

df_inventory2['FECHA AL'] = pd.to_datetime(df_inventory2['FECHA AL'], format='%d/%m/%Y')
# max_date = df_inventory2['FECHA AL'].max()
# df_inventory2 = df_inventory2[df_inventory2['FECHA AL'] == max_date]
df_inventory2['FECHA AL'] = df_inventory2['FECHA AL'].dt.strftime('%d/%m/%Y')
df_inventory2['CODIGO'] = df_inventory2['CODIGO'].str.lower()

df_predicciones['date'] = pd.to_datetime(df_predicciones['date'], format='%Y-%m-%d')
df_predicciones['date'] = df_predicciones['date'].dt.strftime('%d/%m/%Y')

merged_df = df_predicciones.merge(
    df_inventory2[['FECHA AL', 'CODIGO', 'STOCK']], 
    how='left', 
    left_on=['articulo'], 
    right_on=['CODIGO']
)

merged_df['STOCK'] = merged_df['STOCK'].fillna(0)

if not back_order.empty:
    merged_df = merged_df.merge(back_order, how='left', on='articulo')
    merged_df['backorder'] = merged_df['backorder'].fillna(0)
else:
    merged_df['backorder'] = 0

merged_df['sobrante'] = np.maximum(merged_df['STOCK'] + merged_df['backorder'] - merged_df['caa_lt'], 0)
merged_df['nueva_compra_sugerida'] = np.maximum(merged_df['caa'] - merged_df['sobrante'], 0)
# merged_df['nueva_compra_sugerida'] = np.ceil(merged_df['nueva_compra_sugerida']).astype(int)
merged_df['nueva_compra_sugerida'] = np.ceil(merged_df['nueva_compra_sugerida']).fillna(0).astype(int)

merge_columns = merged_df[['articulo', 'nueva_compra_sugerida', 'caa', 'backorder']].copy()

dffinal2 = dffinal2.merge(merge_columns, how='left', on='articulo')
dffinal2['compra_sugerida'] = dffinal2['nueva_compra_sugerida'].fillna(0)
dffinal2['backorder'] = dffinal2['backorder'].fillna(0)

mask = dffinal2['demanda_mensual'] != 0
dffinal2.loc[mask, 'meses_proteccion'] = (
    dffinal2.loc[mask, 'meses_proteccion'] * 
    (dffinal2.loc[mask, 'compra_sugerida'] / dffinal2.loc[mask, 'demanda_mensual'])
)

columns_to_drop = ['codigo', 'nueva_compra_sugerida', 'caa']
for col in columns_to_drop:
    if col in dffinal2.columns:
        dffinal2 = dffinal2.drop(columns=[col])

def finalize_processing(self) -> None:
dffinal2 = dffinal2.rename(columns={'caa_x': 'compras_recomendadas'})
dffinal2 = dffinal2.drop_duplicates()
# dffinal2['compras_recomendadas'] = dffinal2['compras_recomendadas'].apply(lambda x: math.ceil(x / 50) * 50)
dffinal2['compras_recomendadas'] = dffinal2['compras_recomendadas'].fillna(0).apply(lambda x: math.ceil(x / 50) * 50)
dffinal2['costo_compra'] = dffinal2['monto_usd'] * dffinal2['compras_recomendadas']

df1_final_filled = df1_final.fillna(0)
df1_final_grouped = df1_final_filled.groupby(['articulo', 'fuente_suministro']).agg({
    'ingreso_usd_sin_recomendacion': 'sum',
    'ingreso_usd_con_recomendacion': 'sum'
}).reset_index()

dffinal2 = dffinal2.merge(
    df1_final_grouped[['articulo', 'fuente_suministro']], 
    how='left', 
    on='articulo'
)

dffinal2 = dffinal2.merge(
    df_rfm, 
    left_on='articulo',
    right_on='sku',
    how='left'
)
dffinal2['rfm'] = dffinal2['rfm'].fillna(0).astype(int)

df1_final_grouped['ganancia_oportunidad'] = (
    df1_final_grouped['ingreso_usd_con_recomendacion'] - 
    df1_final_grouped['ingreso_usd_sin_recomendacion']
)

df1_final_grouped_fs = df1_final_grouped.groupby(['fuente_suministro']).agg({
    'ganancia_oportunidad': 'sum'
}).reset_index()

df1_final_grouped_fs = df1_final_grouped_fs.sort_values(
    by='ganancia_oportunidad', 
    ascending=False
).reset_index(drop=True)

df1_final_grouped_fs['hierarchy'] = df1_final_grouped_fs.index + 1

dffinal2 = dffinal2.merge(
    df1_final_grouped_fs[['fuente_suministro', 'hierarchy']], 
    how='left', 
    on='fuente_suministro'
)

# dffinal2['venta_acumulada'] = dffinal2['demanda_mensual'] * dffinal2['monto_usd'] * dffinal2['lt_x']
# dffinal2['deficit'] = dffinal2['venta_acumulada'] - (dffinal2['stock'] + dffinal2['backorder']) * dffinal2['monto_usd']
# dffinal2['deficit'] = dffinal2['deficit'].apply(lambda x: max(x, 0))  # El déficit no puede ser negativo
# dffinal2['urgency'] = dffinal2['deficit'].rank(method='min', ascending=False).fillna(0).astype(int)

dffinal2.loc[dffinal2['demanda_mensual'] < 0, 'demanda_mensual'] = 0
dffinal2.loc[dffinal2['compras_recomendadas'] < 0, 'compras_recomendadas'] = 0
# dffinal2['demanda_mensual'] = dffinal2['demanda_mensual'].fillna(0) 
# dffinal2['compras_recomendadas'] = dffinal2['compras_recomendadas'].fillna(0) 

# Calcular riesgo
dffinal2['holgura'] = dffinal2['stock'] / dffinal2['demanda_mensual']
dffinal2['consumiendo_proteccion'] = (dffinal2['holgura'] < dffinal2['meses_proteccion']).astype(int)
dffinal2['quebro'] = (dffinal2['holgura'] <= 0).astype(int)
dffinal2['va_a_quebrar'] = ((dffinal2['stock'] + dffinal2['backorder']) < dffinal2['demanda_mensual']).astype(int)
dffinal2['verde'] = (
    (dffinal2['quebro'] == 0) & 
    (dffinal2['consumiendo_proteccion'] == 0) & 
    (dffinal2['va_a_quebrar'] == 0)
).astype(int)
dffinal2['amarillo'] = (
    (dffinal2['consumiendo_proteccion'] == 1) & 
    (dffinal2['quebro'] == 0)
).astype(int)
dffinal2['rojo'] = (
    (dffinal2['quebro'] == 1) |
    (dffinal2['va_a_quebrar'] == 1) 
    ).astype(int)
dffinal2['riesgo'] = dffinal2.apply(
    lambda row: 'rojo' if row['rojo'] == 1 else 
                'amarillo' if row['amarillo'] == 1 else 
                'verde',
    axis=1
)

# filtrar solo las importantes para la tabla por fuente de suministro
dffinal2['demanda_mensual_usd'] = dffinal2['demanda_mensual'] * dffinal2['monto_usd']
df_temp = dffinal2.copy()
df_temp = df_temp[(df_temp['rfm']==3) & (df_temp['riesgo']=='rojo')]
df_temp = df_temp.groupby('fuente_suministro').agg(
    recomendacion=('costo_compra', 'sum'),
    demanda_mensual_usd = ('demanda_mensual_usd','sum')
).reset_index()
df_temp2 = dffinal2.groupby('fuente_suministro').agg(
    lead_time=('lt_x', 'first'),
    riesgo=('riesgo', lambda x: x.mode()[0] if not x.mode().empty else None), # moda
).reset_index()
dffinal3 = df_temp2.merge(df_temp, how='left',on='fuente_suministro')
dffinal3['recomendacion'] = dffinal3['recomendacion'].fillna(0).astype(int)
dffinal3['demanda_mensual_usd'] = dffinal3['demanda_mensual_usd'].fillna(0).astype(int)

# filtrar rfm = 3 para tabla de dashboard
# dfdashboard = dffinal2[dffinal2['rfm']==3]

# dar formato
dffinal3['riesgo_color'] = dffinal3['riesgo'].map({
    'verde': '🟢',
    'amarillo': '🟡',
    #'naranja': '🟠',
    'rojo': '🔴'
})
dffinal2 = dffinal2[[
    'articulo','fuente_suministro','stock','compras_recomendadas','demanda_mensual','meses_proteccion',
    'riesgo','lt_x','mean_margen','ultima_fecha','monto_usd',
    'ultima_compra','costo_compra','rfm','backorder'
]]
dfdashboard = dffinal2[[
    'articulo','fuente_suministro','stock','backorder','rfm','riesgo',
    'monto_usd','ultima_compra','compras_recomendadas','costo_compra'
]]
dffinal3 = dffinal3[[
    'fuente_suministro',
    'lead_time',
    'recomendacion',
    'demanda_mensual_usd'
]]
# columns mapping
display_columns = {
    'articulo': 'Artículo',
    'fuente_suministro': 'Fuente Suministro',
    'stock': 'Inventario',
    'backorder': 'Backorder',
    'compras_recomendadas': 'Compras Recomendadas',
    'rfm':'Importancia RFM',
    'riesgo': 'Alerta',
    'monto_usd': 'Monto USD',
    'ultima_compra': 'Última Compra',
    'demanda_mensual': 'Demanda Mensual',
    'lt_x': 'Lead Time',
    'mean_margen': 'Margen',
    'meses_proteccion': 'Meses proteccion',
    'ultima_fecha': 'Ultima fecha',
    'costo_compra': 'Recomendacion USD'
    }
display_columns_fuente = {
    'fuente_suministro': 'Fuente de Suministro',
    'lead_time': 'Lead Time',
    'recomendacion': 'Recomendacion USD',
    'demanda_mensual_usd': 'Demanda Mensual USD',
    'mean_margen': 'Margen Promedio'
}

dfdashboard = dfdashboard.rename(columns=display_columns)
dffinal2 = dffinal2.rename(columns=display_columns)
dffinal3 = dffinal3.rename(columns=display_columns_fuente)

In [ ]:
##########################

In [159]:
file_path = 'data/raw/catusita/Data de venta 01.01.21 a 06.12.24.xls'

df_catusita = pd.read_excel(file_path, sheet_name="Sheet1")
lista_columnas = df_catusita.columns.tolist()

excel_file = pd.ExcelFile(file_path)
list_hojas = excel_file.sheet_names[1:]

for hoja in list_hojas:
    df_catusita_hoja = pd.read_excel(file_path, sheet_name=hoja, header=None)
    df_catusita_hoja.columns = lista_columnas
    callable = pd.concat([df_catusita, df_catusita_hoja], ignore_index=True)

In [165]:
df_catusita['Fecha'].max()

Timestamp('2024-12-06 00:00:00')

In [168]:
df_products['fecha'].max()

'2024-11-30'